In [2]:
import overpass
from pprint import pprint
import requests
from openrouteservice import client

ovp_api = overpass.API(timeout=600)
openroute_api_key = '5b3ce3597851110001cf62480279a95227aa4e5f9ccf7cfa946e69f3'
opr_client = client.Client(key=openroute_api_key) 

## Overpass API

In [ ]:
q = [
    # supermarkets (around location)
    'node["shop"="supermarket"](around:2000.0, 51.782065, 19.459279);',
    
    # cafes or fast_foods with polish website (around location)
    'node["amenity"~"cafe|fast_food"]["website"~".pl/$"](around:2000.0, 51.782065, 19.459279);',
    
    # banks far away from police (around location)
    'nwr["amenity"="police"](around:2000, 51.782065, 19.459279)->.polices;\
    nwr["amenity"="bank"](around:1000, 51.782065, 19.459279)->.banks;\
    nwr.banks(around.polices:1000)->.banksNearPolices;\
    (.banks; - .banksNearPolices;);out;',
    
    # hotels near lakes (around location)
    'nwr["tourism"="hotel"](around:100000, 51.782065, 19.459279)->.hotels;\
    nwr["water"="lake"]["name"~"^Jezioro"](around:101000, 51.782065, 19.459279)->.lakes;\
    nwr.hotels(around.lakes:1000);out;'
]

response = ovp_api.get(q[1])
pprint(response)

## OpenRouteService

In [34]:
params = {
    'locations': [(19.459279, 51.782065)],
    'profile': 'driving-car',
    'range_type': 'time',
    'range': [10*60],
    'smoothing': 1.
}
resp1 = opr_client.isochrones(**params)


In [30]:
params = {
    'locations': [(19.459279, 51.782065)],
    'profile': 'foot-walking',
    'range_type': 'time',
    'range': [5*60],
    'smoothing': 0.75
}
resp2 = opr_client.isochrones(**params)

In [31]:
pprint(resp2)

{'bbox': [19.453986, 51.778876, 19.463749, 51.785312],
 'features': [{'geometry': {'coordinates': [[[19.453986, 51.781136],
                                             [19.453991, 51.781119],
                                             [19.455125, 51.779415],
                                             [19.457105, 51.778876],
                                             [19.461211, 51.778936],
                                             [19.461775, 51.779075],
                                             [19.462681, 51.779512],
                                             [19.46291, 51.779802],
                                             [19.463728, 51.78118],
                                             [19.463749, 51.781669],
                                             [19.463623, 51.782006],
                                             [19.46124, 51.784589],
                                             [19.460546, 51.784791],
                                             [19.45

In [54]:
def makePolyStr(opr_coords):
    coord_str = '"'
    for coord in opr_coords:
        coord_str += str(coord[1]) + ' ' + str(coord[0]) + ' '
    coord_str = coord_str.rstrip(' ')
    coord_str += '"'
    return '(poly:{})'.format(coord_str)

def makeAroundLineStr(around_dist_m, opr_coords):
    coord_str = ''
    for coord in opr_coords:
        coord_str += ',' + str(coord[1]) + ',' + str(coord[0])
    return '(around:{}{})'.format(around_dist_m, coord_str)

print(makePolyStr(resp2['features'][0]['geometry']['coordinates'][0]))
print(makeAroundLineStr(100, resp2['features'][0]['geometry']['coordinates'][0]))

(poly:"51.781136 19.453986 51.781119 19.453991 51.779415 19.455125 51.778876 19.457105 51.778936 19.461211 51.779075 19.461775 51.779512 19.462681 51.779802 19.46291 51.78118 19.463728 51.781669 19.463749 51.782006 19.463623 51.784589 19.46124 51.784791 19.460546 51.785312 19.458504 51.785308 19.458335 51.785104 19.457738 51.78435 19.456605 51.784042 19.456262 51.783315 19.455689 51.781343 19.454086 51.781136 19.453986")
(around:100,51.781136,19.453986,51.781119,19.453991,51.779415,19.455125,51.778876,19.457105,51.778936,19.461211,51.779075,19.461775,51.779512,19.462681,51.779802,19.46291,51.78118,19.463728,51.781669,19.463749,51.782006,19.463623,51.784589,19.46124,51.784791,19.460546,51.785312,19.458504,51.785308,19.458335,51.785104,19.457738,51.78435,19.456605,51.784042,19.456262,51.783315,19.455689,51.781343,19.454086,51.781136,19.453986)


In [58]:
opr_coords = resp2['features'][0]['geometry']['coordinates'][0]
dist_m = 100
ovp_query_1 = '(nwr["shop"="convenience"]{};\
nwr["shop"="convenience"]{};)->.shops;\
nwr["tourism"="apartment"]{}->.apartments;\
nwr.apartments(around.shops:{});\
out;'.format(makePolyStr(opr_coords), makeAroundLineStr(dist_m, opr_coords), makePolyStr(opr_coords), dist_m)
print(ovp_query_1)
ovp_resp_1 = ovp_api.get(ovp_query_1)
pprint(ovp_resp_1)

(nwr["shop"="convenience"](poly:"51.781136 19.453986 51.781119 19.453991 51.779415 19.455125 51.778876 19.457105 51.778936 19.461211 51.779075 19.461775 51.779512 19.462681 51.779802 19.46291 51.78118 19.463728 51.781669 19.463749 51.782006 19.463623 51.784589 19.46124 51.784791 19.460546 51.785312 19.458504 51.785308 19.458335 51.785104 19.457738 51.78435 19.456605 51.784042 19.456262 51.783315 19.455689 51.781343 19.454086 51.781136 19.453986");nwr["shop"="convenience"](around:100,51.781136,19.453986,51.781119,19.453991,51.779415,19.455125,51.778876,19.457105,51.778936,19.461211,51.779075,19.461775,51.779512,19.462681,51.779802,19.46291,51.78118,19.463728,51.781669,19.463749,51.782006,19.463623,51.784589,19.46124,51.784791,19.460546,51.785312,19.458504,51.785308,19.458335,51.785104,19.457738,51.78435,19.456605,51.784042,19.456262,51.783315,19.455689,51.781343,19.454086,51.781136,19.453986);)->.shops;nwr["tourism"="apartment"](poly:"51.781136 19.453986 51.781119 19.453991 51.779415 19

## TagInfo API

In [16]:
payload = {
    'key': 'amenity',
    'page': 1,
    'rp': 30,
    'sortname': 'count_all',
    'sortorder': 'desc'    
}

r = requests.get('https://taginfo.openstreetmap.org/api/4/key/values', payload)
vals = [e['value'] for e in r.json()['data']]
pprint(vals)

['parking',
 'bench',
 'place_of_worship',
 'restaurant',
 'school',
 'parking_space',
 'fuel',
 'cafe',
 'waste_basket',
 'fast_food',
 'bank',
 'bicycle_parking',
 'shelter',
 'toilets',
 'pharmacy',
 'post_box',
 'recycling',
 'kindergarten',
 'drinking_water',
 'hospital',
 'bar',
 'atm',
 'grave_yard',
 'post_office',
 'pub',
 'vending_machine',
 'hunting_stand',
 'clinic',
 'police',
 'doctors']
